# Cleanup Image Backend - Google Colab Setup

This notebook runs the backend on a free Google Colab GPU (T4) and exposes it via ngrok.

## Instructions:
1. Requires an Ngrok Authtoken (free account at [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken))
2. Copy your token and paste it when prompted below.

In [ ]:
# 1. Clone Repository
!git clone https://github.com/yudilee/cleanup-image.git
%cd cleanup-image/backend

In [ ]:
# 2. Install System Dependencies
!apt-get update && apt-get install -y libgl1-mesa-glx libhl-amdgpu-pro

In [ ]:
# 3. Install Python Dependencies
!pip install -r requirements.txt
!pip install pyngrok uvicorn nest_asyncio

In [ ]:
# 4. Install PixelHacker Dependencies (Optional - for high quality model)
# Note: This might downgrade some packages, but it's usually fine in Colab
!pip install diffusers==0.30.2 transformers==4.40.0 accelerate==0.34.0 peft

In [ ]:
# 5. Setup Ngrok & Run Server
import getpass
from pyngrok import ngrok, conf
import nest_asyncio
import uvicorn
import sys

# Patch asyncio for Colab
nest_asyncio.apply()

# Get Ngrok Token
print("Please enter your Ngrok Authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken):")
token = getpass.getpass('Ngrok Authtoken: ')

if token:
    conf.get_default().auth_token = token
    
    # Kill existing tunnels
    ngrok.kill()

    # Start tunnel
    public_url = ngrok.connect(8000)
    print(f"\n\n🚀 Backend is running! Copy this URL into your Frontend ENV or Code: {public_url}\n\n")
    
    # Run the server
    # Note: We import main here to run it via python, assuming main.py is in current dir
    # If main.py expects to be run as a module, we might need adjustments.
    # But for simplicity, we can just run uvicorn programmatically.
    
    # To ensure imports work correctly, we append the path
    sys.path.append('.')
    
    # Run Uvicorn
    uvicorn.run("main:app", host="127.0.0.1", port=8000)
else:
    print("❌ Error: Ngrok token is required to expose the server.")